In [3]:
from datasets import load_dataset
data_files = {"train": '/raid/juliana/depsign/data/train_initial.csv', "validation": '/raid/juliana/depsign/data/dev_data.csv'}
raw_datasets = load_dataset(
        "csv",
        data_files=data_files
    )

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-f3c0c2a7e7025318/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-large')
tokenizer2 = AutoTokenizer.from_pretrained('microsoft/deberta-v3-large')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
max_seq_length = 512
head_limit = 128
head_tail_dataset = {}

o_tokenized_dataset = tokenizer(raw_datasets['train']['text data'], padding=False)
tokenized_dataset = tokenizer(raw_datasets['train']['text data'], padding=True)
for key in tokenized_dataset.keys():
    head_tail_dataset[key] = []
    token_to_find = tokenizer.pad_token_id if key == 'input_ids' else 0
    for tokens in tokenized_dataset[key]:
        if token_to_find in tokens: 
            size = tokens.index(token_to_find)
            if size > max_seq_length:
                head_tail_dataset[key].append(tokens[:head_limit]+ tokens[size-max_seq_length+head_limit:size])
            else:
                head_tail_dataset[key].append(tokens[:max_seq_length])
        else:
            head_tail_dataset[key].append(tokens[:max_seq_length])

In [104]:
index = 20
print(head_tail_dataset['input_ids'][index], head_tail_dataset['attention_mask'][index], len(head_tail_dataset['input_ids'][index]), len(head_tail_dataset['attention_mask'][index]))

print(o_tokenized_dataset['input_ids'][index], o_tokenized_dataset['attention_mask'][index], len(o_tokenized_dataset['input_ids'][index]), len(o_tokenized_dataset['attention_mask'][index]))

[0, 100, 17, 27, 548, 554, 2410, 10, 1040, 804, 59, 127, 23630, 6453, 4832, 38, 1034, 4909, 7005, 24, 4, 407, 444, 5907, 34, 4, 3791, 24, 17, 27, 29, 45, 10, 5674, 961, 16, 6908, 236, 7, 1798, 30, 143, 839, 4, 125, 38, 33, 7, 120, 24, 66, 7421, 4, 38, 64, 17, 27, 90, 4719, 42, 7, 2185, 5988, 4, 178, 38, 33, 5907, 1493, 7, 1067, 7, 4, 38, 64, 17, 27, 90, 3116, 11, 10, 25694, 1303, 38, 810, 284, 2609, 24, 4, 13308, 38, 216, 3127, 8832, 99, 38, 17, 27, 119, 164, 149, 4, 152, 16, 70, 38, 33, 4, 1437, 3401, 905, 4909, 1166, 127, 1040, 8, 888, 4686, 19, 24, 4, 2780, 162, 45, 28, 5, 129, 65, 54, 34, 5, 6453, 4, 38, 56, 1085, 1493, 4, 20, 232, 115, 23, 513, 492, 162, 42, 4, 20, 44, 48, 4651, 76, 17, 46, 115, 23, 513, 492, 162, 42, 4, 38, 240, 42, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [67]:
tokens[:10]+tokens[size+1-40+10:size+1]

[0,
 38065,
 2838,
 13,
 127,
 1508,
 7,
 33,
 10,
 9336,
 21,
 15983,
 14,
 353,
 4,
 978,
 38,
 95,
 2067,
 734,
 24,
 17,
 27,
 29,
 10,
 7735,
 169,
 7,
 1760,
 8,
 619,
 6,
 53,
 24,
 2653,
 10,
 828,
 2340,
 4,
 2]

In [62]:
tokens[size+1-10:size+1]

[619, 6, 53, 24, 2653, 10, 828, 2340, 4, 2]

In [39]:
tokens.index(tokenizer.sep_token_id)

ValueError: 2 is not in list

In [38]:
tokenizer.sep_token_id

2

In [ ]:
tokenized_dataset[]

In [27]:
import pandas as pd
token_size = []
for tokens in tokenized_dataset['input_ids']:
    token_size.append(len(tokens))
print(sum(token_size)/len(token_size))

5753.0


In [28]:
pd.Series(token_size).describe(percentiles= [0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999])

count    7201.0
mean     5753.0
std         0.0
min      5753.0
25%      5753.0
50%      5753.0
75%      5753.0
90%      5753.0
95%      5753.0
99%      5753.0
99.9%    5753.0
max      5753.0
dtype: float64

In [35]:
sentence1_key = 'text data'
sentence2_key = None
m_head_limit = 0.25
padding = True
max_seq_length = 512

def preprocess_function(examples):
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        
        head_limit = m_head_limit
        if head_limit is None:
            result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
        else:
            if isinstance(head_limit, float):
                head_limit = round(max_seq_length*head_limit)
            result = {
                'input_ids': [],
                'attention_mask': []
            }
            tokenized_dataset = tokenizer(*args, padding=True)
            token_to_find = tokenizer.sep_token_id
            for tokens, attention_mask in zip(tokenized_dataset['input_ids'], tokenized_dataset['attention_mask']):
                size = tokens.index(token_to_find) + 1
                if size > max_seq_length:
                    result['input_ids'].append(tokens[:head_limit]+ tokens[size-max_seq_length+head_limit:size])
                    result['attention_mask'].append(attention_mask[:head_limit]+ attention_mask[size-max_seq_length+head_limit:size])
                else:
                    result['input_ids'].append(tokens[:max_seq_length])
                    result['attention_mask'].append(attention_mask[:max_seq_length])


        # Map labels to IDs (not necessary for GLUE tasks)
        #if label_to_id is not None and "label" in examples:
        #    result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        result["label"] = examples["label"]
        return result

In [36]:
raw_datasets2 = raw_datasets.map(
    preprocess_function,
    batched=True,
)

Map:   0%|          | 0/7201 [00:00<?, ? examples/s]

Map:   0%|          | 0/3245 [00:00<?, ? examples/s]

In [37]:
round(max_seq_length*0.25)

128

In [49]:
i = raw_datasets2['train']['input_ids'][3568]
a = raw_datasets2['train']['attention_mask'][3568]
print(i, a, len(i), len(a))

[0, 34256, 20207, 13878, 9, 127, 23384, 4832, 38, 581, 386, 30, 584, 42, 34, 57, 402, 38, 348, 2425, 7, 109, 13, 29402, 122, 4, 1590, 127, 86, 19, 6943, 38, 174, 2185, 114, 38, 655, 303, 127, 169, 66, 9, 24, 14, 38, 74, 618, 127, 676, 98, 14, 2085, 951, 66, 89, 54, 21, 95, 101, 162, 115, 30282, 2115, 42, 8, 120, 357, 23, 513, 95, 10, 410, 828, 4, 1437, 407, 42, 16, 162, 3565, 127, 527, 552, 19, 383, 14, 269, 1147, 162, 11, 1989, 4, 50118, 2709, 144, 9, 127, 301, 38, 74, 33, 1602, 2185, 25, 10, 1372, 6, 7168, 621, 4, 2497, 284, 6, 205, 964, 6, 205, 301, 11, 937, 4, 280, 18, 45, 7, 224, 127, 301, 143, 430, 87, 608, 402, 38, 74, 33, 3680, 3776, 4, 1308, 11098, 21, 117, 1181, 11, 127, 2228, 797, 98, 38, 342, 127, 1503, 15, 99, 21, 6, 8, 14, 21, 8106, 4, 1437, 50118, 3226, 13540, 23996, 7022, 4, 12606, 152, 94, 65, 429, 2369, 10, 410, 7735, 6, 53, 2128, 63, 5, 650, 8, 8372, 383, 101, 42, 14, 64, 269, 244, 951, 4, 38, 21, 608, 10, 319, 9, 751, 173, 148, 2569, 2608, 377, 8, 21, 5705, 2498, 10

In [50]:
print(o_tokenized_dataset['input_ids'][3568])

[0, 34256, 20207, 13878, 9, 127, 23384, 4832, 38, 581, 386, 30, 584, 42, 34, 57, 402, 38, 348, 2425, 7, 109, 13, 29402, 122, 4, 1590, 127, 86, 19, 6943, 38, 174, 2185, 114, 38, 655, 303, 127, 169, 66, 9, 24, 14, 38, 74, 618, 127, 676, 98, 14, 2085, 951, 66, 89, 54, 21, 95, 101, 162, 115, 30282, 2115, 42, 8, 120, 357, 23, 513, 95, 10, 410, 828, 4, 1437, 407, 42, 16, 162, 3565, 127, 527, 552, 19, 383, 14, 269, 1147, 162, 11, 1989, 4, 50118, 2709, 144, 9, 127, 301, 38, 74, 33, 1602, 2185, 25, 10, 1372, 6, 7168, 621, 4, 2497, 284, 6, 205, 964, 6, 205, 301, 11, 937, 4, 280, 18, 45, 7, 224, 127, 301, 21, 481, 9, 143, 3722, 20802, 4, 19282, 4249, 58, 10, 891, 9, 498, 38, 206, 38, 56, 10, 891, 4315, 15798, 19, 10941, 1385, 6943, 4, 96, 262, 212, 4978, 89, 58, 10, 367, 688, 147, 38, 21, 182, 614, 13582, 4, 22079, 1033, 9, 45, 12365, 11, 6, 103, 7735, 1026, 9, 11902, 4, 38, 2145, 145, 23, 127, 7014, 268, 77, 951, 54, 21, 402, 9, 10, 1441, 376, 81, 8, 553, 162, 114, 38, 21, 30103, 6, 8, 14, 6649,

In [47]:
[len(i) for i in o_tokenized_dataset['input_ids']].index(max([len(i) for i in o_tokenized_dataset['input_ids']]))

3568

In [17]:
raw_datasets2['train']['input_ids'][0]

[0,
 38065,
 2838,
 13,
 127,
 1508,
 7,
 33,
 10,
 9336,
 683,
 5,
 44,
 48,
 4030,
 2041,
 17,
 46,
 2157,
 965,
 17,
 27,
 90,
 89,
 5988,
 4832,
 38,
 218,
 17,
 27,
 90,
 216,
 59,
 1268,
 1493,
 6,
 53,
 38,
 17,
 27,
 119,
 10,
 410,
 828,
 3915,
 14,
 38,
 17,
 27,
 890,
 213,
 124,
 7,
 145,
 16658,
 11,
 10,
 367,
 360,
 86,
 50,
 402,
 4,
 1426,
 76,
 6,
 38,
 1381,
 45,
 7,
 33,
 143,
 9336,
 29,
 13,
 5,
 386,
 9,
 954,
 4,
 83,
 9198,
 158,
 360,
 423,
 6,
 38,
 2263,
 159,
 9701,
 4,
 38,
 938,
 17,
 27,
 90,
 5,
 276,
 13,
 14,
 1445,
 76,
 4,
 3105,
 454,
 719,
 6,
 147,
 38,
 21,
 15983,
 14,
 353,
 4,
 978,
 38,
 95,
 2067,
 734,
 24,
 17,
 27,
 29,
 10,
 7735,
 169,
 7,
 1760,
 8,
 619,
 6,
 53,
 24,
 2653,
 10,
 828,
 2340,
 4,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 

In [7]:
label_list = raw_datasets["train"].unique("label")

In [11]:
label_list

['moderate', 'not depression', 'severe']

In [13]:
from sklearn.utils.class_weight import compute_class_weight
computed_class_weights = compute_class_weight(
    class_weight = 'balanced',
    classes = ['not depression', 'moderate', 'severe'],
    y = raw_datasets['train']['label'])
computed_class_weights

array([0.87126437, 0.65261918, 3.12543403])

In [10]:
import torch
torch.tensor(computed_class_weights)

tensor([0.6526, 0.8713, 3.1254], dtype=torch.float64)

In [3]:
import math
def generate_dist_matrix(num_labels):
    dist_matrix = []
    for i in range(num_labels):
        dist_label = []
        for j in range(num_labels):
            dist_label.append(abs(i-j))
        dist_matrix.append(dist_label)
    return dist_matrix

In [6]:
generate_dist_matrix(3)

[[0, 1, 2], [1, 0, 1], [2, 1, 0]]